In [9]:
import sys
from io import StringIO
import numpy as np
import pandas as pd

# TODO: 
# - Store results in csv if optioned
# - Add option for directory

def extract_table(contents: str, keyword: str, delim: str ='\n\n\n') -> str:
    """
    Extracts strings consisting of only table values specific for .dat files
    """
    # Find keyword index
    start_index = contents.find(keyword)
    if start_index == -1:
        raise ValueError(f"Keyword '{keyword}' not found in file.")
    
    start_index += len(keyword)
    
    # Start remaining text after keyword
    remaining_text = contents[start_index:]
    
    # Find 2 occurrences of delim
    positions = []
    search_start = 0
    
    count = 0
    while count != 2:
        pos = remaining_text.find(delim, search_start)
        if pos == -1:
            break
        if count == 0:
            table_start_index = pos + 3
        elif count == 1:
            table_end_index = pos
            
        count += 1
        search_start = pos +3
    
    try:
        extracted_text = remaining_text[table_start_index:table_end_index]
    except:
        extracted_text = remaining_text[table_start_index:]
    
    return extracted_text

def get_mode_table_df(contents: str):
    keyword = 'E I G E N V A L U E    O U T P U T'
    table = extract_table(contents, keyword)

    mode_table_df = pd.read_csv(
        StringIO(table),
        sep=r'\s+',
        header=None,
        names=[
            'mode_no',
            'eigenvalue',
            'freq (rad/time)',
            'freq (cycles/time)',
            'generalized mass',
            'composite model damping'
        ]
    )

    mode_table_df = mode_table_df[['mode_no', 'freq (cycles/time)']].copy()
    mode_table_df.rename(columns = {'freq (cycles/time)': 'freq'}, inplace=True)

    return mode_table_df

def get_mode_df(contents, mode_number):
    num_str = str(mode_number)
    keyword = f"E I G E N V A L U E    N U M B E R{num_str.rjust(6, ' ')}"
    table_content_start = contents.find(keyword)
    table_content = contents[table_content_start + len(keyword):]

    table = extract_table(table_content, 'U3', delim='\n  \n')

    max_index = table.find('MAX') 
    if max_index == -1:
        raise Exception("Incorrect file format")
    table = table[:max_index].rstrip()

    # print(table)
    mode_df = pd.read_csv(
        StringIO(table),
        sep=r'\s+',
        header=None,
        names=[
            'node_no',
            'U1',
            'U2',
            'U3'
        ]
    )

    return mode_df    

def get_U_inplane(mode_df):
    U1_sum = mode_df['U1'].sum()
    U3_sum = mode_df['U3'].sum()
    return np.sqrt(U1_sum**2 + U3_sum**2).item()

def get_U_outplane(mode_df):
    return mode_df['U2'].sum().item()

def check_limit(l1, l2, threshold=300):
    errors = 0
    for a in l1:
        for b in l2:
            if abs(a - b) < threshold:
                print(f"Failed because inplane ({a}) is {threshold} within outplane ({b})")
                errors += 1
    
    print(f"Found {errors} error(s)")

In [10]:
filename = "C346RS_frnt_rotor_modal_separation_10Jun25.dat"

In [11]:
with open(filename, "r") as file:
    contents = file.read()

In [281]:
mode_table_df = get_mode_table_df(contents)
max_modes = mode_table_df['mode_no'].max().item()

In [287]:
def get_proportions(mode_df, n):
    # only use positive y values
    mode_df = mode_df[mode_df['U2'] >= 0].copy()
    
    mode_df['sq_x'] = mode_df['U1']**2
    mode_df['sq_y'] = mode_df['U2']**2
    mode_df['sq_z'] = mode_df['U3']**2
    
    sumsq_x = mode_df['sq_x'].sum()
    sumsq_y = mode_df['sq_y'].sum()
    sumsq_z = mode_df['sq_z'].sum()

    total_energy = sumsq_x + sumsq_y + sumsq_z

    oop = (sumsq_y) / total_energy
    ip = (sumsq_x + sumsq_z) / total_energy

    print(f"Mode {n} has OOP: {oop*100}%, IP: {ip*100}%.")
    return oop*100, ip*100, sumsq_x, sumsq_y, sumsq_z

In [266]:
def get_average_disp(mode_df, n):
    return mode_df['abs_sum_xyz'].sum() / len(mode_df)

In [183]:
outplane_modes = [29, 36, 45, 48, 65, 71, 94, 103]
inplane_modes = [32, 33, 46, 47, 68, 69, 99, 100]

In [252]:
# 4 parameters: node -> threshold, lower_p_thres, upper_p_thres,
# ip -> proportion

In [288]:
inplane = []
for mode_number in range(1, max_modes+1):
    mode_df = get_mode_df(contents, mode_number)
    oop, ip, x, y, z = get_proportions(mode_df, mode_number)
    print(f"x+z: {x+z}\n")

Mode 1 has OOP: 97.55958524998591%, IP: 2.440414750014082%.
x+z: 11618.018781334853

Mode 2 has OOP: 97.55958867759088%, IP: 2.4404113224091124%.
x+z: 11617.220758330033

Mode 3 has OOP: 99.4802581833743%, IP: 0.5197418166257033%.
x+z: 4844.427307921258

Mode 4 has OOP: 97.90446372556245%, IP: 2.0955362744375536%.
x+z: 11160.043013197188

Mode 5 has OOP: 97.90447879408939%, IP: 2.095521205910602%.
x+z: 11160.086661468324

Mode 6 has OOP: 1.4835888910720667e-06%, IP: 99.9999985164111%.
x+z: 426337.1737930838

Mode 7 has OOP: 17.052412038016946%, IP: 82.94758796198306%.
x+z: 239461.87864167368

Mode 8 has OOP: 17.116658003327892%, IP: 82.88334199667212%.
x+z: 238620.37730478757

Mode 9 has OOP: 97.72889480833173%, IP: 2.2711051916682683%.
x+z: 12834.227257352792

Mode 10 has OOP: 97.72195282139336%, IP: 2.2780471786066325%.
x+z: 12874.305166803637

Mode 11 has OOP: 8.979240841200822%, IP: 91.02075915879918%.
x+z: 195052.1317453395

Mode 12 has OOP: 8.974101953169585%, IP: 91.025898046830

In [296]:
inplane = []
for mode_number in range(1, max_modes+1):
    mode_df = get_mode_df(contents, mode_number)
    oop, ip, x, y, z = get_proportions(mode_df, mode_number)
    if ip > 96 and (x+z) > 100000:
        inplane.append(mode_number)

Mode 1 has OOP: 97.55958524998591%, IP: 2.440414750014082%.
Mode 2 has OOP: 97.55958867759088%, IP: 2.4404113224091124%.
Mode 3 has OOP: 99.4802581833743%, IP: 0.5197418166257033%.
Mode 4 has OOP: 97.90446372556245%, IP: 2.0955362744375536%.
Mode 5 has OOP: 97.90447879408939%, IP: 2.095521205910602%.
Mode 6 has OOP: 1.4835888910720667e-06%, IP: 99.9999985164111%.
Mode 7 has OOP: 17.052412038016946%, IP: 82.94758796198306%.
Mode 8 has OOP: 17.116658003327892%, IP: 82.88334199667212%.
Mode 9 has OOP: 97.72889480833173%, IP: 2.2711051916682683%.
Mode 10 has OOP: 97.72195282139336%, IP: 2.2780471786066325%.
Mode 11 has OOP: 8.979240841200822%, IP: 91.02075915879918%.
Mode 12 has OOP: 8.974101953169585%, IP: 91.0258980468304%.
Mode 13 has OOP: 89.26918764844231%, IP: 10.730812351557686%.
Mode 14 has OOP: 97.16850542555535%, IP: 2.8314945744446463%.
Mode 15 has OOP: 97.15967610944371%, IP: 2.8403238905562946%.
Mode 16 has OOP: 84.04405420179867%, IP: 15.95594579820132%.
Mode 17 has OOP: 82.1

In [297]:
inplane

[6, 32, 33, 46, 47, 68, 69, 100, 101, 129]

In [294]:
for mode_number in inplane:
    mode_df = get_mode_df(contents, mode_number)
    oop, ip, x, y, z = get_proportions(mode_df, mode_number)
    if mode_number in inplane_modes:
        print("CORRECT")
    print(f"x+z: {x+z}")
    print(f"y: {y}\n")

Mode 60 has OOP: 2.133229357836311%, IP: 97.86677064216369%.
x+z: 57111.81983061801
y: 1244.8823021615422

Mode 61 has OOP: 1.7863266757113143%, IP: 98.21367332428868%.
x+z: 67159.7930263064
y: 1221.5135200372642

Mode 70 has OOP: 0.9691565074584988%, IP: 99.03084349254148%.
x+z: 2473.739668322053
y: 24.20903238588602

Mode 75 has OOP: 0.5360215702609913%, IP: 99.46397842973902%.
x+z: 81631.18999079196
y: 439.9188463192909

